# Exploiting Collaborative Preferences

Publication information: Martins, D. M. L., Vossen, G., & Maleszka, M. (2018, October). Supporting Online Data Purchase by Preference Recommendation. In 2018 IEEE International Conference on Systems, Man, and Cybernetics (SMC) (pp. 3703-3708). IEEE.

URL: https://ieeexplore.ieee.org/document/8616624

Publication's BibTeX:
```
@INPROCEEDINGS{8616624,
    author={D. M. L. {Martins} and G. {Vossen} and M. {Maleszka}},
    booktitle={2018 IEEE International Conference on Systems, Man, and Cybernetics (SMC)},
    title={Supporting Online Data Purchase by Preference Recommendation},
    year={2018},
    volume={},
    number={},
    pages={3703-3708},
    keywords={commerce;data handling;decision making;electronic commerce;purchasing;recommender systems;online data purchase;data-driven decision-making;organizations;personal requirements;alternative solutions;traditional data commerce;data marketplaces;novice buyers;intelligent decision support approach;preference recommendation strategy;useful data offerings;Semiotics;Neurons;Decision making;Self-organizing feature maps;Data models;Organizations},
    doi={10.1109/SMC.2018.00627},
    ISSN={1062-922X},
    month={Oct},
}
```

### Configuring notebook

In [ ]:
cd ..

In [ ]:
cd ..

In [ ]:
import sys, os
sys.path.append('..\\sada')
sys.path.append('..\\decision')

In [ ]:
import pandas as pd
from datamanagement.dataaccessobject import DataAccessObject, Dataset
from decision.clause import *
from decision.criterion import Criterion
from decision.somselector import SomSelector
from sada.decisionsada import DecisionSADA

### Loading car dataset

In [ ]:
DAO = DataAccessObject()

In [ ]:
dataset = DAO.get_car_dataset()

In [ ]:
dataset.preprocessed_data.columns

### Creating SADA

In [ ]:
sada = DecisionSADA(dataset)

### Creating the decision database

In [ ]:
def print_preferences(preferences):
    for pref in preferences:
        print(pref.to_string())

class DecisionEntry(object):
    def __init__(self, preferences, criteria, optimal_candidates, success):
        self.preferences = preferences
        self.criteria = criteria
        self.optimal_candidates = optimal_candidates
        self.success = success
    
def build_historical_data(sada):
    b_1 = DecisionEntry(preferences = [DiadicClause('Volkswagen', Operation.EQUALS, 1), DiadicClause('mpg', Operation.GREATER_THAN_EQUALS, 0.098)],
                        criteria = [Criterion('price', maximize=False, weight=0.6), Criterion('mpg', maximize=True, weight=0.4)],
                        optimal_candidates = None, success = True)
    
    b_2 = DecisionEntry(preferences = [DiadicClause('price', Operation.LESS_THAN_EQUALS, 0.231193), DiadicClause('Sporty', Operation.EQUALS, 1), DiadicClause('Origin', Operation.EQUALS, 0)],
                        criteria = [Criterion('price', maximize=False, weight=0.6), Criterion('horsepower', maximize=True, weight=0.4)],
                        optimal_candidates = None, success = True)

    b_3 = DecisionEntry(preferences = [DiadicClause('driver_passenger', Operation.EQUALS, 1), DiadicClause('Front', Operation.EQUALS, 1)],
                        criteria = [Criterion('price', maximize=False, weight=0.6), Criterion('passenger_capacity', maximize=True, weight=0.4)],
                        optimal_candidates = None, success = 0)

    b_4 = DecisionEntry(preferences = [DiadicClause('price', Operation.LESS_THAN_EQUALS, 0.01), DiadicClause('Small', Operation.EQUALS, 1)],
                        criteria = [Criterion('price', maximize=False, weight=0.6), Criterion('mpg', maximize=True, weight=0.4)],
                        optimal_candidates = None, success = 0)

    b_5 = DecisionEntry(preferences = [DiadicClause('price', Operation.LESS_THAN_EQUALS, 0.03), DiadicClause('Small', Operation.EQUALS, 1)],
                        criteria = [Criterion('length', maximize=False, weight=0.6), Criterion('mpg', maximize=True, weight=0.4)],
                        optimal_candidates = None, success = 1)

    b_6 = DecisionEntry(preferences = [DiadicClause('Rear', Operation.EQUALS, 1), DiadicClause('Sporty', Operation.EQUALS, 1), DiadicClause('passenger_capacity', Operation.EQUALS, 0)],
                        criteria = [Criterion('RPM', maximize=True, weight=0.6), Criterion('horsepower', maximize=True, weight=0.4)],
                        optimal_candidates = None, success = 1)

    b_7 = DecisionEntry(preferences = [DiadicClause('Compact', Operation.EQUALS, 1), DiadicClause('passenger_capacity', Operation.GREATER_THAN_EQUALS, 0.5), DiadicClause('Rear', Operation.EQUALS, 1)],
                        criteria = [Criterion('mpg', maximize=True, weight=0.6), Criterion('price', maximize=False, weight=0.4)],
                        optimal_candidates = None, success = 0)

    b_8 = DecisionEntry(preferences = [DiadicClause('luggage_capacity', Operation.GREATER_THAN_EQUALS, 0.875)],
                        criteria = [Criterion('mpg', maximize=True, weight=0.6), Criterion('fuel_tank_capacity', maximize=True, weight=0.4)],
                        optimal_candidates = None, success = 1)
    
    previous_decisions = [b_1, b_2, b_3, b_4, b_5, b_6, b_7, b_8]
    
    for b in previous_decisions:
        selected, optimal = sada.get_recommendations(b.preferences, b.criteria)
        b.optimal_candidates = optimal
    
    return previous_decisions

In [ ]:
previous_decisions = build_historical_data(sada)

### Current buyer

In [ ]:
#preferences=[DiadicClause('price', Operation.LESS_THAN_EQUALS, 0.08), #6000
#        DiadicClause('horsepower', Operation.GREATER_THAN_EQUALS, 0.4)] #150

#criteria = [Criterion('price', maximize=False, weight=0.6),
#            Criterion('horsepower', maximize=True, weight=0.4)]

preferences=[DiadicClause('Compact', Operation.EQUALS, 0), DiadicClause('Origin', Operation.EQUALS, 0), DiadicClause('price', Operation.LESS_THAN_EQUALS, 0.08), DiadicClause('mpg', Operation.LESS_THAN_EQUALS, 0.41)]

criteria = [Criterion('mpg', maximize=True, weight=0.5), Criterion('price', maximize=False, weight=0.5)]


current_buyer = DecisionEntry(preferences=preferences, criteria=criteria, 
                              optimal_candidates=[], success=False)

### Select and evaluate cantidates

In [ ]:
selected, current_buyer.optimal_candidates = sada.get_recommendations(current_buyer.preferences, current_buyer.criteria)

In [ ]:
selected

In [ ]:
current_buyer.optimal_candidates[['make', 'type', 'price', 'horsepower', 'mpg']]

### Finding the most similar buyers from the decision database

In [ ]:
def create_som_vector(buyer, vector_size):
    vector = []
    for i in range(vector_size):
        if len(buyer.optimal_candidates) > i:
            vector.append(int(buyer.optimal_candidates.index[i]))
        else:
            vector.append(-1)
    return vector

def create_som_dataset(previous_decisions, vector_size):
    som_data = []
    for p in previous_decisions:
        vector = create_som_vector(p, vector_size)
        som_data.append(vector)

    som_dataset = Dataset(som_data, pd.DataFrame(som_data))
    return som_dataset

def get_jaccard_most_similar(previous_decisions, current_buyer):
    current_optimal = set(current_buyer.optimal_candidates['make'].values)
    history = {}
    for i in range(len(previous_decisions)):
        history[i] = set(previous_decisions[i].optimal_candidates['make'])
    similarity = []
    for k,v in history.items():
        jaccard = len(current_optimal & v)/len(current_optimal | v)
        similarity.append(jaccard)
    
    most_similar = similarity.index(max(similarity))
    complementary_candidates = history[most_similar] - (current_optimal & history[most_similar])
    return most_similar, complementary_candidates

In [ ]:
som_selector = SomSelector(som_size=(3, 3), num_iterations=1000)

In [ ]:
current_buyer_vector = create_som_vector(current_buyer, 5)
som_dataset = create_som_dataset(previous_decisions, 5)

In [ ]:
similar_indices = som_selector.select(current_buyer_vector, som_dataset, num_selected_items=3)

In [ ]:
most_similar_decisions = [previous_decisions[i] for i in similar_indices]

### Creating reject preference short-term memory

In [ ]:
rejected_preferences = []

### Calculating preference weights

In [ ]:
tau_increasing_factor = 1

In [ ]:
all_preferences = []

In [ ]:
for dec in most_similar_decisions:
    for pref in dec.preferences:
        if not pref in all_preferences:
            weight = 0
            if (not pref in current_buyer.preferences) and (not pref in rejected_preferences) and dec.success:
                weight += tau_increasing_factor
            entry = (pref, weight)
            all_preferences.append(entry)

In [ ]:
# Sort preferences
all_preferences = [pref for pref in sorted(all_preferences, key=lambda pref: pref[1], reverse=True)]

In [ ]:
relevant_preferences = [pref[0] for pref in all_preferences if pref[1] > 0]
print_preferences(relevant_preferences)

In [ ]:
print_preferences(current_buyer.preferences)

### Recommend the preference with highest weight that generates a list of diverse optimal offers

In [ ]:
recommended_preferences = []

In [ ]:
for pref in relevant_preferences:
    if pref in recommended_preferences:
        continue
    else:
        extended_preferences = [pref]
        extended_preferences.extend(current_buyer.preferences)
        sel, opt = sada.get_recommendations(query=extended_preferences, criteria=current_buyer.criteria)
        A = set(current_buyer.optimal_candidates.index.tolist())
        B = set(opt.index.tolist())
        if A != B: # Check wheater the preference produces a change into the optimal candidates retrieved by SADA
            recommended_preferences.append(pref)
        #if set(current_buyer.optimal_candidates.index.tolist()) != set(opt.index.tolist()):
            #recommended_preferences.append(pref)

In [ ]:
print_preferences(recommended_preferences)

### Update buyer preferences

In [ ]:
current_buyer.preferences.append(recommended_preferences[0])

In [ ]:
selected_recommended, optimal_recommended = sada.get_recommendations(query=current_buyer.preferences, criteria=current_buyer.criteria)

In [ ]:
selected_recommended

In [ ]:
optimal_recommended[['make', 'type', 'price', 'horsepower', 'mpg']]